In [10]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

In [11]:
#import dataset from data.csv file
df = pd.read_csv('Dummy_Month.csv')
df.head()

,Unnamed: 0,Bay_id,Status,Hour,Minutes,DayOfWeek,Date
0,0,1001,Unoccupied,0,0,Wednesday,2021-08-04
1,1,1002,Present,0,0,Wednesday,2021-08-04
2,2,1003,Present,0,0,Wednesday,2021-08-04
3,3,1004,Unoccupied,0,0,Wednesday,2021-08-04
4,4,1005,Present,0,0,Wednesday,2021-08-04


In [12]:
#The amount of observations (rows) and characteristics(columns) in a dataset
print ('The Total Rows in the dataset = ' + str(df.shape[0]))
print ('The Total Columns in the dataset = ' + str(df.shape[1]))

The Total Rows in the dataset = 13440
The Total Columns in the dataset = 7


In [148]:
Bay_ID = (df['Bay_id'] == 1001)
Multiple_Bay_ID = (df['Bay_id'].isin([1001, 1003, 1005]))
Parking_Status_Present = (df['Status'] == 'Present')
Parking_Status_Unoccupied = (df['Status'] == 'Unoccupied')
Day_Of_Week = (df['DayOfWeek'] == 'Friday')
Date = (df['Date'] == '2021-08-06')
Hour = (df['Hour'] == 12)

## Current Parking Availability

In [161]:
parking_pres = df.loc[Multiple_Bay_ID & Parking_Status_Present & Date].groupby('Status').DayOfWeek.value_counts().unstack(0)
parking_pres = parking_pres.reset_index()
parking_pres.columns = ['Day', 'Parking_Occupied']

CountStatus = df.loc[Multiple_Bay_ID & Parking_Status_Unoccupied & Date].groupby('Status').DayOfWeek.value_counts().unstack(0)
CountStatus = CountStatus.reset_index()
CountStatus.columns = ['Day', 'Parking_Unoccupied']

Parkingdf = pd.merge(parking_pres, CountStatus, on=['Day'])
Parkingdf = pd.DataFrame(Parkingdf)

Parkingdf = Parkingdf.eval('Sum_Parking = Parking_Unoccupied + Parking_Occupied')
Parking_SUM = Parkingdf.iloc[0,3]
Parking_SUM = int(round(Parking_SUM))
Parkingdf

,Day,Parking_Occupied,Parking_Unoccupied,Sum_Parking
0,Friday,133,155,288


In [162]:
Day_of_Choice = Parkingdf.iloc[0,0]
Day_of_Choice

'Friday'

In [163]:
Today = Parkingdf.iloc[0,1]
Today

133

In [164]:
parking_diff = Parkingdf.iloc[0,2]
parking_diff

155

In [165]:
Parking_SUM = Parkingdf.iloc[0,3]
Parking_SUM = int(round(Parking_SUM))
Parking_SUM

288

In [166]:
diff = Today - (parking_diff)

In [167]:
fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = Today,
    mode = "gauge+number+delta",
    title = {'text': f"Parking Availability on {Day_of_Choice}"},
    delta = {'reference': diff},
    gauge = {'axis': {'range': [None, Parking_SUM]},
             'bar': {'color': "firebrick"},
             'steps' : [{'range': [0, Parking_SUM], 'color': "ghostwhite"}],
             'threshold' : {'line': {'color': "black", 'width': 5}, 'thickness': 0.75, 'value': Today}}))

fig.add_annotation(text=f"There are {parking_diff} empty parking spaces", 
                   showarrow=False, 
                   yshift=-210,
                   font=dict(size=22, color="royalblue"))

fig.show()